In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [2]:
# Paths and settings
feat_dir  = '/home/mtang/vslib/mlpc2025_Team_Laborer/MLPC2025_classification/audio_features'
label_dir = '/home/mtang/vslib/mlpc2025_Team_Laborer/MLPC2025_classification/labels'
split_csv = '/home/mtang/vslib/mlpc2025_Team_Laborer/notebooks/Task3/tang/data_split_tang.csv'
subset    = ['Speech','Dog Bark','Siren','Rain','Car']
selected_feats = ['mfcc','embeddings','contrast','flatness','bandwidth','melspectrogram']

# Load split info
split_df   = pd.read_csv(split_csv)
train_files = split_df[split_df['split']=='train']['filename'].tolist()
val_files   = split_df[split_df['split']=='val']['filename'].tolist()

# Define scaler+PCA pipeline
feature_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=47))
])

def load_data(files):
    """
    Load raw selected features and binary labels for 'subset' classes.
    Returns X_raw (n_frames × D_total), y (n_frames × 5).
    """
    X_list, y_list = [], []
    for fn in files:
        arr_feat = np.load(os.path.join(feat_dir, fn.replace('.mp3','.npz')))
        arr_lab  = np.load(os.path.join(label_dir, fn.replace('.mp3','_labels.npz')))
        # Stack selected features per frame
        mats = [arr_feat[f] for f in selected_feats]
        X_file = np.concatenate(mats, axis=1)       # (T, D_total)
        # Build y_file: each class 1 if votes ≥ half
        votes = np.vstack([
            (arr_lab[cls].sum(axis=1) >= np.ceil(arr_lab[cls].shape[1]/2))
            for cls in subset
        ]).T                                       # (T,5)
        X_list.append(X_file)
        y_list.append(votes.astype(int))
    X_raw = np.vstack(X_list)
    y     = np.vstack(y_list)
    return X_raw, y

# Load raw data
X_train_raw, y_train = load_data(train_files)
X_val_raw,   y_val   = load_data(val_files)

# Fit/transform
X_train = feature_pipeline.fit_transform(X_train_raw)
X_val   = feature_pipeline.transform(X_val_raw)


In [ ]:
# Define hyperparameter grids
svm_Cs   = [0.01, 0.1, 1, 10, 100]
knn_ks   = [1, 3, 5, 7, 9]
dt_depth = [None, 5, 10, 20]

# Containers for results
svm_res, knn_res, dt_res = [], [], []

# SVM
for C in svm_Cs:
    clf = OneVsRestClassifier(SVC(kernel='rbf', C=C, random_state=47))
    clf.fit(X_train, y_train)
    y_tr = clf.predict(X_train)
    y_va = clf.predict(X_val)
    svm_res.append({
        'param': C,
        'f1_train': f1_score(y_train, y_tr, average='macro'),
        'f1_val':   f1_score(y_val,   y_va, average='macro')
    })

# KNN
for k in knn_ks:
    clf = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=k))
    clf.fit(X_train, y_train)
    y_tr = clf.predict(X_train)
    y_va = clf.predict(X_val)
    knn_res.append({
        'param': k,
        'f1_train': f1_score(y_train, y_tr, average='macro'),
        'f1_val':   f1_score(y_val,   y_va, average='macro')
    })

# Decision Tree
for d in dt_depth:
    clf = OneVsRestClassifier(DecisionTreeClassifier(max_depth=d, random_state=47))
    clf.fit(X_train, y_train)
    y_tr = clf.predict(X_train)
    y_va = clf.predict(X_val)
    dt_res.append({
        'param': 'None' if d is None else d,
        'f1_train': f1_score(y_train, y_tr, average='macro'),
        'f1_val':   f1_score(y_val,   y_va, average='macro')
    })


In [ ]:
# Plot
def plot_curve(results, name, xlabel):
    df = pd.DataFrame(results)
    plt.figure(figsize=(4,3))
    plt.plot(df['param'], df['f1_train'], label='Train F1', marker='o')
    plt.plot(df['param'], df['f1_val'],   label='Val F1',   marker='o')
    plt.title(f'{name}')
    plt.xlabel(xlabel)
    plt.ylabel('Macro-F1')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# SVM
plot_curve(svm_res, 'SVM: F1 vs C', 'C (log scale)')
# KNN
plot_curve(knn_res, 'KNN: F1 vs k', 'n_neighbors')
# Decision Tree
plot_curve(dt_res, 'Decision Tree: F1 vs max_depth', 'max_depth')
